# Lab 2: Custom Common-Envelope (CE) Step

When running **POSYDON**, the core–envelope boundary can only be defined using three fixed hydrogen mass fraction thresholds:  
$$
X_H = 0.01,\; 0.1,\; \text{or}\; 0.3
$$

### Motivation
---
In some cases, however, we may want to explore other values of the hydrogen mass fraction to define the core boundary.  
This gives us more flexibility in studying how different assumptions affect the outcome of common-envelope evolution.

### Goal of this Lab
---
In this lab, we will:
1. Develop a **custom CE step** that allows us to use **any arbitrary value** of the hydrogen mass fraction to determine the core–envelope boundary.  
2. Introduce and use our functions (`calculate_binding_energy` and `calculate_E_lambda_CE`) to calculate the post-CE outcome.  
3. Apply this custom CE step to a **population of double neutron stars (DNS)**.  

For this exercise, begin by downloading a **population of DNS systems**.  
We will then **re-run this population** using our custom CE step and compare the resulting DNS distributions when different definitions of the core boundary are used.  


We don't want to edit the source code so we will need to make our own step. In the Lab folder there is a python file custom_CE_step.py that will substitute the existing CE_step in the POSYDON flow.

 We will edit the CE and complete the pieces that are missing. There should be comments in the code that will indicate where you need to fill with code in `.py`. 
 
 We are mostly going to edit only these threes that we will add code to are : 
 1. `CEE_simple_alpha_prescription()`
 2. `calculate_lamda_CE()`
 3. `calculate_binding_energy()`



### Step 1

Take a look at the function `CEE_simple_alpha_prescription` in the custom_CE_step.py. 

The function takes as inputs the `donor` star and the `comp_star`, these are object of the class `Single_star`. 

In our first step we need to define the variables of pre CE masses, radii and the states of the two stars as `m1_i`, `radius1`,`state1_i` for the donor and `m2_i`, `radius2`,`state2_i`. These values are atrributes of the object Single star. Look at the file `POSYDON/posydon/binary_evol/singlestar.py` on how these attributes are defined. 

<div class='alert alert-warning'>
    
**Hint:** You can find them in `STARPROPERTIES`
    
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
    m1_i = donor.mass
    radius1 = 10**donor.log_R
    state1_i = donor.state
    m2_i = comp_star.mass
    radius2 = 10**comp_star.log_R
```
    
</details>
</div>

### Step 2
We want to define two variables related to the binary period and the alpha_CE that has been given from ini the file: `period_i` and `alpha_CE`

Look in the binary_star.py how to access the binary orbital period from the binary Object.

Look at the initialization function of class for the alpha_CE. 

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
    period_i = binary.orbital_period
    alpha_CE = self.common_envelope_efficiency 
```
    
</details>
</div>

### Step 3 

The final profile of the stars can be access from the attribute `single.profile`. 

The outcome will be a long array as we saw in Lab 1. 

Since we built our function to handle pandas DataFrames we are lso going load the profiles in two variables `donor_prof` and `comp_prof` as pandas DataFrames.

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
    donor_prof = pd.DataFrame(donor.profile)
    comp_prof = pd.DataFrame(comp_star.profile)
```
    
</details>
</div>

### Step 4 

Take the two functions from Lab 1 and paste them where the function are defined in the .py file.

 We also need our `calculate_lamdbda_CE` function to return two more variables related to the donor's core and donor's radius as found from our arbitrary boundary condition. 
 
  Make the necessary changes in the `calculate_lamdbda_CE` to return the mass and radius of the donor's core from the star's profile.

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
    def calculate_binding_energy(self,core_definition_H_fraction,star,common_envelope_alpha_thermal = 1):

        Grav_energy = 0.0
        U_i = 0.0

        radius = np.array(star.radius)
        internal_energy = np.array(star.energy)
        mass = np.array(star.mass) 
        print()
        dm = np.concatenate((-1 * np.diff(mass),[mass[-1]]))

        zones = star[star.x_mass_fraction_H > core_definition_H_fraction].index
        zone_core = zones[-1]
        for i in range(zone_core):
            Grav_energy_i = (-const.standard_cgrav * mass[i]
                                   * const.Msun * dm[i]*const.Msun
                                   / (radius[i]*const.Rsun))
            # integral of gravitational energy as we go deeper into the star
            Grav_energy = Grav_energy + Grav_energy_i
            U_i = U_i + internal_energy[i]*dm[i]*const.Msun

        # binding energy of the enevelope equals its gravitational energy +
        # an a_th fraction of its internal energy
        Ebind_i = Grav_energy + common_envelope_alpha_thermal * U_i
        return Ebind_i
    
    
    
    def calculate_lambda_CE(self,core_definition_H_fraction,star,common_envelope_alpha_thermal = 1):
        zones = star[star.x_mass_fraction_H > core_definition_H_fraction].index
        zone_i = zones[-1]
        E_bind = self.calculate_binding_energy(core_definition_H_fraction,star,common_envelope_alpha_thermal = 1)
        mass = star.mass
        radius = star.radius
        M_donor = mass[0]
        M_core = mass[zone_i]
        M_envelope = M_donor - M_core
        R_core = radius[zone_i]
        R = radius[0]
        lambda_CE = - const.standard_cgrav *(M_donor*const.Msun)*(M_envelope*const.Msun)/(R*const.Rsun*E_bind)
        return lambda_CE,M_core,R_core
```
    
</details>
</div>

### Step 5
Use the equation below to calculate the **binding energy** of the donor star.  
Write this calculation next to `ebind_i =` in the file `custom_CE_step.py`.  


$$
E_{\rm bind} = - \frac{G M_{\rm donor} \, M_{\rm env}}{\lambda \, R_{\rm donor}}
$$


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
ebind_i = (-const.standard_cgrav / lambda1_CE
                       * (m1_i * const.Msun * (m1_i - mc1_i) * const.Msun)
                       / (radius1 * const.Rsun))
```
    
</details>
</div>

For a **double common envelope (CE)**, the total binding energy is the **sum of the individual stars’ binding energies**.  

Below the condition `if double_CE:`, calculate the binding energy of the secondary star and add it to the total binding energy `ebind_i`.  


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
if double_CE:
    ebind_i += (-const.standard_cgrav / lambda2_CE
                            * (m2_i * const.Msun * (m2_i - mc2_i) * const.Msun)
                            / (radius2 * const.Rsun))
```

</details>
</div>

### Step 6
Next, calculate the **initial separation** of the binary before it underwent CE evolution.  

We only have access to the binary's **orbital period**, not the separation stored in the binary object.  
Fortunately, you don’t need to write a new function for this.  

In `posydon/utils/common_functions.py`, there is already a function that converts **orbital period → separation**.  

1. Find that function in the file.  
2. Check which units the result is returned in.  
3. Use it to calculate the **pre-CE separation** in **cgs units**.  

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
     separation_i = const.Rsun * cf.orbital_separation_from_period(
            period_i, m1_i, m2_i)   # in cgs units
```
    
</details>
</div>

### Step 7

Now we have all the parameters needed to calculate the **orbital energy** required to eject the envelope.  
Using the equations below, we can solve for the **post-CE binary separation** \(a_f\).  

The orbital energy of the system is given by:  

$$
E_{\rm orb} =  \frac{G M_{\rm donor} M_2}{2 a_0}
$$


$$
\Delta E_{\rm orb} = E_{\rm orb}^{f} - E_{\rm orb}^{i} 
= - \frac{G M_{\rm core} M_2}{2 a_f} + \frac{G M_{\rm donor} M_2}{2 a_0}
$$


and 

$$
    E_{bind} = a_{CE} \, \Delta E_{orb}
$$


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
        eorb_i = (-0.5 * const.standard_cgrav * m1_i * const.Msun
                  * m2_i * const.Msun / separation_i)
        

        eorb_postCEE = eorb_i + ebind_i/alpha_CE

        separation_postCEE = (-0.5 * const.standard_cgrav * mc1_i * const.Msun
                              * mc2_i * const.Msun / eorb_postCEE)
```
    
</details>
</div>

---

Now, we are ready to use our step and we need to introduce to the flow. We will do that by editing the .ini file. Copy the ini file from `posydon/popsyn/population_params_default.ini` in your working directory and rename it as `my_population_params.ini`

---
In your ini file under `step_CE` we have an option to point to our file for this step. This can be done by changing the option `absolute_import =` to point to the file with our own step. Edit the absolute_import to contain the path to our file and the class name of our CE_step. 

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
    absolute_import = ['path/to/your/custom_CE_step.py','StepCEE']
```
    
</details>
</div>

Before we start running the population we need to change the `interpolation_method` to the `nearest_neighbour` in all the steps that have that option. The reason behind this choice has to do with the access to final profiles of the stars (You will learn more about this later in the week). 

### Re-evolving a population
---

To test if everything is running fine we can try and re-run a subsection of the DNS population in our notebooks for debbuging. Go to the ini file and change the number of binaries to 10 and then execute the code bellow in your notebook. 

In [ ]:
from posydon.popsyn.synthetic_population import BinaryPopulation
from posydon.binary_evol.simulationproperties import SimulationProperties
from posydon.popsyn.io import binarypop_kwargs_from_ini
from posydon.utils.common_functions import convert_metallicity_to_string
import argparse
from posydon.popsyn.io import simprop_kwargs_from_ini
from posydon.config import PATH_TO_POSYDON_DATA
import os

ini_kw = binarypop_kwargs_from_ini('./my_population_params.ini')
sim_kw = simprop_kwargs_from_ini('./my_population_params.ini')
#print(ini_kw)
ini_kw['metallicity'] = 1.0

# This is just building the path to where our populations are saved for this lab.
data_path = os.path.join(os.path.dirname(PATH_TO_POSYDON_DATA), "2025_school_data/populations/DNS_pops")

ini_kw['file_name'] = os.path.join(data_path, 'solar_DNS.h5')
sim_prop = SimulationProperties(**sim_kw)
poprun = BinaryPopulation(population_properties=sim_prop,**ini_kw)



"\nfrom posydon.popsyn.synthetic_population import BinaryPopulation\nfrom posydon.binary_evol.simulationproperties import SimulationProperties\nfrom posydon.popsyn.io import binarypop_kwargs_from_ini\nfrom posydon.utils.common_functions import convert_metallicity_to_string\nimport argparse\nfrom posydon.popsyn.io import simprop_kwargs_from_ini\nini_kw = binarypop_kwargs_from_ini('./my_population_params.ini')\nsim_kw = simprop_kwargs_from_ini('./my_population_params.ini')\n#print(ini_kw)\nini_kw['metallicity'] = 1.0\nini_kw['file_name'] = './solar_DNS.h5'\nsim_prop = SimulationProperties(**sim_kw)\npoprun = BinaryPopulation(population_properties=sim_prop,**ini_kw)\n"

In [ ]:
poprun.evolve(from_hdf=True,tqdm=True)
poprun.save('./test_solar_DNS.h5' )

If no errors occured during your run, inspect your population to make sure that everything looks okay. If it takes you more than a few minutes to debug your code take a look at the solution file : `custom_CE_step_sol.py` for additional help. 

__If your errors persist use the solution as your custom step instead to save time.__

Now let's run our entire population:
 - Change the value for the boundary core and alpha from the values you picked in the spreadsheet in your ini file.
 - Change the number of binaries from 10 to **400**. 
 - Go to the `run_job.sh` file and replace the paths in the export of the `PATH_TO_POSYDON` and `PATH_TO_POSYDON_DATA` with your own paths.  
 - Make sure you also put your own email in the `--mail-user` option.
 - Comment out these lines in the ini file: 
  ```  
scalar_names = [
'interp_class_HMS_HMS',
'interp_class_CO_HMS_RLO',
'interp_class_CO_HeMS',
'interp_class_CO_HeMS_RLO',
'mt_history_HMS_HMS',
'mt_history_CO_HMS_RLO',
'mt_history_CO_HeMS',
'mt_history_CO_HeMS_RLO',]
```
- Submit your job to the cluster by running the command ```sbatch run_job.sh```

---

**After submitting your job, make sure that it started running in the queue**. You can check the status of your job by this command. 

```
squeue -u <username>
```

__If any issues come up, ask right away a TA  to help you out.__

The job should take 10-15 minutes. While we wait for our job to run we will continue with the rest of the Lab to calculate the merger times of the solar type DNS in a prerun population that is provided in the lab's folder, named : `solar_DNS.py`.

---

### Calculate the merger times of DNS
The inspiral time of double neutron stars can be estimated by: 
___

$$
    \tau_{merger} = 10^7 yr \, P_{orb,h}^{8/3} \, \big ( \frac{M}{M_\odot} \big )^{-2/3} \big ( \frac{\mu}{M_\odot} \big )^{-1} (1 - e^2)^{7/2}
$$


We will calculate the merger times from the fiducial population and then compare it with the merger times from the population produced with our custom CE step. 



### Step 1 
Import the population using pandas arrays and load the history. 

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python 
    pop= pd.read_hdf("./solar_DNS.h5", key = 'history')


```

</details>
</div>


Look at the history of the first binary with `pop.loc[0]`. The line after the event of `CC2` it's where the DNS is newly form. We want to extract the information and binary properties at this point of the binary's evolution and not at the final step, as the binary's orbital period is further evolved as a double compact object. 


### Step 2 

Extract the binary properties of the newly formed DNS from the history.  

We want to locate the line immediately **after** the event `CC2`. This can be done using `np.where()`, which returns the `iloc` indices where `event == 'CC2'`. The indices that follow those positions correspond to the newly formed DNS systems.  

Additionally, we need to ensure that both stars have indeed become NS. To do this, we add a final filter that checks whether the state of **both stars** is equal to `'NS'`.  



<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint (click to reveal)</summary></b>
    
   The `iloc` indexes containg the systems we want can be found by: 

```python 
    np.where(pop.event == 'CC2')[0] + 1
```

</details>
</div>


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python 
    newly_formed_systems = pop.iloc[np.where(pop.event.values == "CC2")[0]+1]
    formed_DNS = newly_formed_systems[(newly_formed_systems.S1_state == 'NS') & (newly_formed_systems.S2_state == 'NS')]
```
    
</details>
</div>

### Step 3 
Instead of writing our own function for the merger time, **POSYDON** already provides one.  
It includes a function that calculates the inspiral time of compact objects given their orbital period, eccentricity, and masses.  

To find it, look in the file `common_functions.py`.  

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python 
    from posydon.utils.common_functions import inspiral_timescale_from_orbital_period
```
    
</details>
</div>

### Step 4
Now let’s use the function `inspiral_timescale_from_orbital_period`.  
We need to iterate over the `formed_DNS` systems and calculate the inspiral time (`t_inspiral`).  

The **merger time** is then given by the time of the newly formed DNS plus its inspiral time.  

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python 
    t_inspiral = []
    for i in range(len(formed_DNS)):
        t = inspiral_timescale_from_orbital_period(
            formed_DNS.S1_mass.iloc[i],
            formed_DNS.S2_mass.iloc[i],
            formed_DNS.orbital_period.iloc[i],
            formed_DNS.eccentricity.iloc[i]
        )
        t_inspiral.append(t)

    t_inspiral = np.array(t_inspiral)
    t_mergers = t_inspiral + np.array(formed_DNS.time)/1e6
```
    
</details>
</div>

### Step 5

Combine the above steps into a single function.  
This function will take a population DataFrame and return the **merger times of only DNS systems**.  


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python 
    def DNS_merger_times(pop):
        newly_formed_systems = pop.iloc[np.where(pop.event.values == "CC2")[0]+1]
        formed_DNS = newly_formed_systems[(newly_formed_systems.S1_state == 'NS') & (newly_formed_systems.S2_state == 'NS')]
        t_inspiral = []
        for i in range(len(formed_DNS)):
            t = inspiral_timescale_from_orbital_period(
                formed_DNS.S1_mass.iloc[i],
                formed_DNS.S2_mass.iloc[i],
                formed_DNS.orbital_period.iloc[i],
                formed_DNS.eccentricity.iloc[i]
            )
            t_inspiral.append(t)

        t_inspiral = np.array(t_inspiral)
        t_inspiral = np.array(t_inspiral)
        t_mergers = t_inspiral + np.array(formed_DNS.time)/1e6
        return t_mergers
```

</details>
</div>

---
Hopefully by now your population has finished its run. You can confirm it by checking on the status of your job. 
___ 
### Step 6 
    
Use the code bellow to plot a histogram of the pre-run DNS population.


In [ ]:
#Histogram of the mergers 
t_mergers = DNS_merger_times(pop)
Hubble_time_Myr = 1.38e4 
plt.figure(figsize=(8,6))
plt.hist(np.log10(t_mergers), bins=50, color="steelblue", edgecolor="black",alpha = 0.4)
plt.xlabel(r"$\log_{10}(t_{\rm merger} \; [\mathrm{Myr}])$")
plt.ylabel("# of NS mergers")
plt.title("DNS Merger Times")
plt.axvline(np.log10(Hubble_time_Myr), color="red", linestyle="--", linewidth=2,
            label="Hubble time (13.8 Gyr)")
plt.show()

--- 

### Step 7 

Overplot the merger times from your population. How do they compare ? Discuss your results with your rest of your group. 

Notice that since we only evolved the first 400 binaries we need to plot only first 400 merger times from our pre-run population.

In [6]:
rerun_pop= pd.read_hdf("./rerun_solar_DNS.h5", key = 'history')
t_mergers = DNS_merger_times(pop)[0:400]
t_mergers_rerun = DNS_merger_times(rerun_pop)
Hubble_time_Myr = 1.38e4 
plt.figure(figsize=(8,6))
plt.hist(np.log10(t_mergers), bins=50, color="steelblue", edgecolor="black",alpha = 0.4, density = True)
plt.hist(np.log10(t_mergers_rerun), bins=50, color="red", edgecolor="black",alpha = 0.4, label = 'rerun', density = True)
plt.xlabel(r"$\log_{10}(t_{\rm merger} \; [\mathrm{Myr}])$")
plt.ylabel("# of NS mergers")
plt.title("DNS Merger Times")
plt.axvline(np.log10(Hubble_time_Myr), color="red", linestyle="--", linewidth=2,
            label="Hubble time (13.8 Gyr)")


'\nrerun_pop= pd.read_hdf("./rerun_solar_DNS.h5", key = \'history\')\nt_mergers = DNS_merger_times(pop)[0:400]\nt_mergers_rerun = DNS_merger_times(rerun_pop)\nHubble_time_Myr = 1.38e4 \nplt.figure(figsize=(8,6))\nplt.hist(np.log10(t_mergers), bins=50, color="steelblue", edgecolor="black",alpha = 0.4, density = True)\nplt.hist(np.log10(t_mergers_rerun), bins=50, color="red", edgecolor="black",alpha = 0.4, label = \'rerun\', density = True)\nplt.xlabel(r"$\\log_{10}(t_{\rm merger} \\; [\\mathrm{Myr}])$")\nplt.ylabel("# of NS mergers")\nplt.title("DNS Merger Times")\nplt.axvline(np.log10(Hubble_time_Myr), color="red", linestyle="--", linewidth=2,\n            label="Hubble time (13.8 Gyr)")\n'